In [1]:
import pandas as pd
import datetime
from datetime import timedelta
import time
import json
import pytz

from azure.servicebus import ServiceBusClient, ServiceBusMessage
from azure.cosmos import CosmosClient
import azure.cosmos.cosmos_client as cosmos_client

from pyspark.sql.types import StructType, ArrayType  
from pyspark.sql.functions import col, when
from delta.tables import *
from pyspark.sql.functions import *
import numpy as np
starttime = time.time()



ERROR:root:Internal Python error in the inspect module.
Below is the traceback from this internal error.


KeyboardInterrupt



In [ ]:
Abnormal = ["NAV", "DTC", "DTO","BKF","BSS","COR",'DAR','DBF','DLF','DLM','EPC','EPR','EPW','EQR','ESB','FLD','HAD','INI','LPS','LRN','NAV','OOS','RCY','REC','STP','UFS','PFO']

In [ ]:
hour_list = ["00","01","02","03","04","05","06","07","08","09","10","11","12","13","14","15","16","17","18","19","20","21","22","23"]
date = datetime.datetime.now(pytz.timezone('Asia/Shanghai')) - timedelta(days=1)
year = str(date.year).zfill(4)
month = str(date.month).zfill(2)
day = str(date.day).zfill(2)
print(date)

In [ ]:
# year ,month , day = '2021' , '05' , '11'

In [ ]:
file_location = f"/{year}/{month}/{day}/"
whole_df = pd.DataFrame()
for i in hour_list:
  df = spark.read.parquet(file_location + i)
  print(file_location + i)
  df_ = df.drop('Version','UnitType','Country_Code','GatewayID','SubID','SIMCardID','MotionSensorLink','CSQ','BatteryVoltage','BackupStatus','LiftState','UnitPowerOff','Floor','Direction','DoorZone','RearDoorStatus','CountCallUp','CountCallDown','Speed','Load','CarDoorLock','FloorDoorLock','SafetyCircuitStatus','DoorProtectorStatus','PassengerExists','DetailDataPerSecond','BlinkMessage')
  df_ = df_.toPandas()
  whole_df = whole_df.append(df_)  
  whole_df.sort_values('ReportTime',inplace=True)
df = whole_df
whole_df

In [ ]:
abnormal_df = df[df['UnitNumber'].isin(df[df['OPMode'].isin(Abnormal)]['UnitNumber'].drop_duplicates())]
abnormal_df['OPMode_shift'] = abnormal_df.groupby('UnitNumber')['OPMode'].shift(1)
print(len(abnormal_df['UnitNumber'].drop_duplicates()))
abnormal_df

In [ ]:
for num , elevator in enumerate(abnormal_df['UnitNumber'].drop_duplicates()):
  elevator_df = abnormal_df[abnormal_df['UnitNumber'] == elevator]
  elevator_df = elevator_df.reset_index(drop = True)
  
  
  start = abnormal_df[(abnormal_df['OPMode'].isin(Abnormal) )& (~abnormal_df['OPMode_shift'].isin(Abnormal) )& (abnormal_df['UnitNumber'] == elevator )]
  start = start['ReportTime'].to_list()
  print('start_list_len: ' + str(len(start)))
  end = abnormal_df[(~abnormal_df['OPMode'].isin(Abnormal) )& (abnormal_df['OPMode_shift'].isin(Abnormal) )& (abnormal_df['UnitNumber'] == elevator )]
  end = end['ReportTime'].to_list()
  print('end_list_len: ' + str(len(end)))
  
  

  var = []
  for start_time in start:
    for end_time_index in range(len(end)):
      if end[end_time_index] > start_time:
        value = end[end_time_index] - start_time
        var.append(value)
        
        # count the Open Status
        if value > 60:
          final = []
          start_index = elevator_df[elevator_df['ReportTime'] == start_time].index[0]
          status = elevator_df.iloc[start_index:start_index + 3,:]['FrontDoorStatus'].to_list()
          final_status = ''
          for i in set(status):
            final_status += str(i) + '_'
          print(elevator , final_status)
          final = pd.DataFrame([elevator,final_status]).T
          final.columns = ['elevator' , 'FrontDoorStatus']
          # save
          result2 = pd.read_csv('/dbfs/FileStore/shared_uploads/lilian.lu@otis.com/OPMode_FrontDoorStatus.csv')
          result2 = pd.concat([result2,final],axis=0).set_index('elevator')
          result2.to_csv('/dbfs/FileStore/shared_uploads/lilian.lu@otis.com/OPMode_FrontDoorStatus.csv')
          
          
        end = end[end_time_index:]
        break
      else:
        continue
  print('完成运行第' + str(num) + "台电梯的数据")
  
#   diction = {'0-22':0,'22-42':0,'42-62':0,'62-120':0,'120以上':0}
#   for value in var:
#     if value <= 22:
#       diction['0-22'] += 1
#     elif 22 < value <= 42:
#       diction['22-42'] += 1
#     elif 42 < value <= 62:
#       diction['42-62'] += 1
#     elif 62 < value <= 120:
#       diction['62-120'] += 1
#     elif value > 120:
#       diction['120以上'] += 1
      
  diction = {'0-30':0,'30-60':0,'60以上':0}
  for value in var:
    if value <= 30:
      diction['0-30'] += 1
    elif 30 < value <= 60:
      diction['30-60'] += 1
    elif value > 60:
      diction['60以上'] += 1
  diction = pd.DataFrame(diction,index=[elevator]).reset_index()
  diction.columns = ['elevator' , '0-30' , '30-60' , '60以上']

  
  # save
  result = pd.read_csv('om/OPMode.csv')
  result = pd.concat([result,diction],axis=0).set_index('elevator')
  result.to_csv('/com/OPMode.csv')
  
  
result

In [ ]:
result.loc['Row_sum'] = result.apply(lambda x: x.sum())
final = pd.DataFrame(result.loc['Row_sum']).T
final.insert(0,'Time','2020/05/11')
final.set_index('Time',inplace=True)
final

In [ ]:
final.div(final.sum(axis=1),axis=0)

In [ ]:
result.to_csv('/dbfs/FileStore/shared_uploads/lilian.lu@otis.com/result2.csv')

In [ ]:
abnormal_df[(abnormal_df['OPMode'] == 'NAV' )& (abnormal_df['OPMode_shift'] != 'NAV' )]  #正常到不正常

abnormal_df[(abnormal_df['OPMode'] != 'NAV' )& (abnormal_df['OPMode_shift'] == 'NAV' )]

In [ ]:
Abnormal = ["NAV", "DTC", "DTO","BKF","BSS","COR",'DAR','DBF','DLF','DLM','EPC','EPR','EPW','EQR','ESB','FLD','HAD','INI','LPS','LRN','NAV','OOS','RCY','REC','STP','UFS','PFO']